# はじめに

Cifarとは、一般物体認識のベンチマークです。CifarはCifar-10とCifar-100に分かれ、数字はクラス数を表します。10は10クラス、100は100クラスをそれぞれ表すわけです。とはいえ、あまり良いCPUを持っていない中で演習をするので、今回はCifar-10で、一般物体認識をしていきましょう。

![Cifar-10の一部](http://cdn-ak.f.st-hatena.com/images/fotolife/a/aidiary/20151014/20151014211729.png)

では、まずCifar-10のデータセットの読み込みから始めましょう。

```
$ wget https://www.cs.toronto.edu/~kriz/cifar-10-python.tar.gz
```

上記のコマンドで、cifar-10のpython versionをダウンロードして展開してみましょう。

```
$ tar xzf cifar-10-python.tar.gz
```

これで作られたcifar-10-batches-pyというディレクトリを使って作業していきます。

```
$ ls cifar-10-batches-py
batches.meta data_batch_1 data_batch_2 data_batch_3 data_batch_4 data_batch_5 readme.html  test_batch
```

ディレクトリの中身を見ると、batch_Xと呼ばれる5つのデータとtest_batch, readmeとなんだかわからないmetaファイルが入っています。readmeを読んでみましょう。どうやら、cifarのページそのままのようです。見るべき部分を抜粋すると、

> The archive contains the files data_batch_1, data_batch_2, ..., data_batch_5, as well as test_batch. Each of these files is a Python "pickled" object produced with cPickle. Here is a Python routine which will open such a file and return a dictionary:

```
def unpickle(file):
    import cPickle
    fo = open(file, 'rb')
    dict = cPickle.load(fo)
    fo.close()
    return dict
```

と書いてあるので、指示通りに作ってみましょう。

In [1]:
import os
import cPickle as pickle

for root, dirs, files in os.walk('cifar-10-batches-py'):
    for fname in files:
        if fname.find('data_batch_') >= 0:
            path = os.path.join(root, fname)
            with open(path, 'rb') as fp:
                data = pickle.load(fp)
                print fname, ":", type(data)
                for k, v in data.items():
                    print '\t', k, ':', type(v)

data_batch_1 : <type 'dict'>
	data : <type 'numpy.ndarray'>
	labels : <type 'list'>
	batch_label : <type 'str'>
	filenames : <type 'list'>
data_batch_2 : <type 'dict'>
	data : <type 'numpy.ndarray'>
	labels : <type 'list'>
	batch_label : <type 'str'>
	filenames : <type 'list'>
data_batch_3 : <type 'dict'>
	data : <type 'numpy.ndarray'>
	labels : <type 'list'>
	batch_label : <type 'str'>
	filenames : <type 'list'>
data_batch_4 : <type 'dict'>
	data : <type 'numpy.ndarray'>
	labels : <type 'list'>
	batch_label : <type 'str'>
	filenames : <type 'list'>
data_batch_5 : <type 'dict'>
	data : <type 'numpy.ndarray'>
	labels : <type 'list'>
	batch_label : <type 'str'>
	filenames : <type 'list'>


さてデータの素性が分かりました。各バッチには、辞書型のオブジェクトが入っていて、その要素にdata, labels, batch_labels, filenamesが入っているらしいですね。では、それぞれの要素を見ていきましょう。今回は、data_batch_1のみを見ていくことにして、また、labels, filenamesにはたくさんのデータが入っていそうなので、最初の10要素だけ表示します。

In [2]:
with open('cifar-10-batches-py/data_batch_1', 'rb') as fp:
    data = pickle.load(fp)
    print 'data: ', data['data'].shape
    print 'labels: ', data['labels'][:10]
    print 'batch_label: ',  data['batch_label']
    print 'filenames: ', data['filenames'][:10]


data:  (10000, 3072)
labels:  [6, 9, 9, 4, 1, 1, 2, 7, 8, 3]
batch_label:  training batch 1 of 5
filenames:  ['leptodactylus_pentadactylus_s_000004.png', 'camion_s_000148.png', 'tipper_truck_s_001250.png', 'american_elk_s_001521.png', 'station_wagon_s_000293.png', 'coupe_s_001735.png', 'cassowary_s_001300.png', 'cow_pony_s_001168.png', 'sea_boat_s_001584.png', 'tabby_s_001355.png']


ここでの出力から、dataには一つ3072次元のデータが10000個入っていて、labelはおそらく0-9の数字が入っているリスト型であり、batch_labelはこのバッチファイルの説明で、pngファイルは、それぞれのエントリのファイル名っぽいことがわかりました。ここで、batch_label, filenamesは学習に必要ないから省くことにします。

まあ、そんなこと調べんでも書いてありますが、

> data -- a 10000x3072 numpy array of uint8s. Each row of the array stores a 32x32 colour image. The first 1024 entries contain the red channel values, the next 1024 the green, and the final 1024 the blue. The image is stored in row-major order, so that the first 32 entries of the array are the red channel values of the first row of the image.
> labels -- a list of 10000 numbers in the range 0-9. The number at index i indicates the label of the ith image in the array data.

さて、本当は、バッチファイル群をぜんぶ結合させて一つの学習セットを作ったほうが良いのでしょうが、次元数がMNISTの4倍ちかくあるので、この時点で4倍の計算量が必要であることがわかります。これを、全部学習するのはサンプルの段階では得策ではありません。とりあえず、batch一つで試してみましょう。ということで、このバッチファイルの中にあるデータに偏りがないかを調べていきます。

In [3]:
for i in xrange(10):
    print '[%d]: ' % i, data['labels'].count(i)

[0]:  1005
[1]:  974
[2]:  1032
[3]:  1016
[4]:  999
[5]:  937
[6]:  1030
[7]:  1001
[8]:  1025
[9]:  981


どうやら、各要素1000個前後に分かれており、それほどのばらつきはなさそうです。ということで、data_batch_1とtest_batchを使って、MLPに与えるデータ・セットを作っていきましょう。

# Cifar10とMLP（「ハリーポッターと賢者の石」的な・・）

さて、ではこれらのデータをこれまたこれまで作ったMLPで学習しましょう。MLPの定義を思い出すと、

```
class MLP(object):
	def __init__(
		self,
		data,
		target,
		n_inputs=784,
		n_hidden=784,
		n_outputs=10,
		gpu=-1
	):
...
```

ということで、入力と出力の次元数を調整できるので、次元数が増えても普通に対応できそうです。なので、後は、これに当てはまるように、データを整形してあげれば良いんですね。まずは、main関数ででてくるdata, targetを作っていきましょう。

In [4]:
import numpy

with open('cifar-10-batches-py/data_batch_1', 'rb') as fp:
    train = pickle.load(fp)

data_train = train['data'].astype(numpy.float32)
data_train /= 255.
target_train = numpy.array(train['labels']).astype(numpy.int32)

with open('cifar-10-batches-py/test_batch', 'rb') as fp:
    test = pickle.load(fp)

data_test = test['data'].astype(numpy.float32)
data_test /= 255.
target_test = numpy.array(test['labels']).astype(numpy.int32)

print 'data_train: ', data_train.shape, data_train[:10]
print 'target_train: ', target_train.shape, target_train[:10]
print 'data_test: ', data_test.shape, data_train[:10]
print 'target_test: ', target_test.shape, target_train[:10]


data_train:  (10000, 3072) [[ 0.23137255  0.16862746  0.19607843 ...,  0.54901963  0.32941177
   0.28235295]
 [ 0.60392159  0.49411765  0.41176471 ...,  0.54509807  0.55686277
   0.56470591]
 [ 1.          0.99215686  0.99215686 ...,  0.32549021  0.32549021
   0.32941177]
 ..., 
 [ 0.10980392  0.11764706  0.12941177 ...,  0.39215687  0.3882353
   0.3764706 ]
 [ 0.52549022  0.51372552  0.50196081 ...,  0.53333336  0.53725493
   0.5411765 ]
 [ 0.49019608  0.43137255  0.40000001 ...,  0.32156864  0.32941177
   0.33725491]]
target_train:  (10000,) [6 9 9 4 1 1 2 7 8 3]
data_test:  (10000, 3072) [[ 0.23137255  0.16862746  0.19607843 ...,  0.54901963  0.32941177
   0.28235295]
 [ 0.60392159  0.49411765  0.41176471 ...,  0.54509807  0.55686277
   0.56470591]
 [ 1.          0.99215686  0.99215686 ...,  0.32549021  0.32549021
   0.32941177]
 ..., 
 [ 0.10980392  0.11764706  0.12941177 ...,  0.39215687  0.3882353
   0.3764706 ]
 [ 0.52549022  0.51372552  0.50196081 ...,  0.53333336  0.53725493
 

ここでは、便利関数train_test_splitを使いませんでした。何故かと言うとlabelデータを見ると既にデータがシャッフルされているからです。厳密に行いたい場合は、再シャッフルをすると良いかもしれませんが、サンプルとして使う分には問題無いでしょう。では、いよいよ、MLPで学習してみましょう。ここからは、MLP.pyのmain関数をほぼほぼコピーしながら進めることができます。

In [5]:
import time
import logging
reload(logging)
logging.basicConfig(level=logging.DEBUG)

from mlp import MLP

data = data_train, data_test
target = target_train, target_test

start_time = time.time()

mlp = MLP(data=data, target=target, gpu=-1, n_inputs=3072, n_outputs=10)
mlp.train_and_test(n_epoch=1)

end_time = time.time()

print("time = {} min".format((end_time - start_time) / 60.0))

INFO:root:epoch 1
INFO:root:train mean loss=2.26340656281, accuracy=0.1670000007
INFO:root:test mean loss=2.02338435531, accuracy=0.24150000006


time = 0.193933769067 min


さて、ここまでが簡単なCifar-10におけるデータの処理方法でした。しかしながら、見てみると分かる通り、識別率は20%前後と一般物体認識はMNISTと比べると、難しい課題であることがわかります。ということで、画像認識に特化していると言われるCNNを使って、Cifar-10の識別をしてみましょう。

# Cifar10とCNN(「ハリーポッターと不死鳥の騎士団」的な・・)

CNNを使って、Cifar10を学習する際の注意点は以下の2つです。

* CNNModelの変更。画像サイズ・チャネル数が変わるので、MLPの展開をしたときの次元数が代わります
* データを2次元から4次元（データ数xチャネル数x縦x横）似直す必要があります

ということで、これらを変更して、Cifar10を学習できるCNNのクラスを作りましょう。というのが本日の課題です。頑張って取り組んでいきましょう！

In [16]:
from chainer import cuda
from chainer import Variable
from chainer import FunctionSet
from chainer import optimizers
import chainer.functions as F

class CNNModel(FunctionSet):
    '''
    1st conv: (32-5+1 , 32-5+1) = (28, 28)
    1st pool: (28/2, 28/2) = (14, 14)
    2nd conv: (14-5+1, 14-5+1) = (10, 10)
    2nd pool: (10/2, 10/2) = (5, 5)
    '''
    def __init__(self, in_channels=3, n_hidden=200, n_outputs=10):
        FunctionSet.__init__(
            self,
            conv1=F.Convolution2D(in_channels, 16, 5),
            conv2=F.Convolution2D(16, 16, 5),
            l3=F.Linear(400, n_hidden),
            l4=F.Linear(n_hidden, n_outputs)
        )

    def forward(self, x_data, y_data, train=True, gpu=-1):
        x, t = Variable(x_data), Variable(y_data)
        h = F.max_pooling_2d(F.relu(self.conv1(x)), ksize=2, stride=2)
        h = F.max_pooling_2d(F.relu(self.conv2(h)), ksize=2, stride=2)
        h = F.dropout(F.relu(self.l3(h)), train=train)
        y = self.l4(h)
        return F.softmax_cross_entropy(y, t), F.accuracy(y, t)

    def predict(self, x_data, gpu=-1):
        x = Variable(x_data)
        h = F.max_pooling_2d(F.relu(self.conv1(x)), ksize=2, stride=2)
        h = F.max_pooling_2d(F.relu(self.conv2(h)), ksize=3, stride=3)
        h = F.dropout(F.relu(self.l3(h)), train=train)
        y = self.l4(h)
        sftmx = F.softmax(y)
        out_data = cuda.to_cpu(sftmx.data)
        return out_data


class CNN(object):
    def __init__(
        self,
        data,
        target,
        in_channels=1,
        n_hidden=100,
        n_outputs=10,
        gpu=-1
    ):
        self.model = CNNModel(in_channels, n_hidden, n_outputs)
        self.model_name = 'cnn.model'

        if gpu >= 0:
            self.model.to_gpu()

        self.gpu = gpu

        self.x_train, self.x_test = data
        self.y_train, self.y_test = target

        self.n_train = len(self.y_train)
        self.n_test = len(self.y_test)

        self.optimizer = optimizers.Adam()
        self.optimizer.setup(self.model)

        self.train_accuracies = []
        self.train_losses = []
        self.test_accuracies = []
        self.test_losses = []

    @property
    def xp(self):
        return cuda.cupy if self.gpu >= 0 else numpy

    def train_and_test(self, n_epoch=20, batchsize=100):
        epoch = 1
        while epoch <= n_epoch:
            logging.info('epoch {}'.format(epoch))

            perm = numpy.random.permutation(self.n_train)
            sum_train_accuracy = 0
            sum_train_loss = 0
            for i in xrange(0, self.n_train, batchsize):
                x_batch = self.xp.asarray(self.x_train[perm[i:i+batchsize]])
                y_batch = self.xp.asarray(self.y_train[perm[i:i+batchsize]])

                real_batchsize = len(x_batch)

                self.optimizer.zero_grads()
                loss, acc = self.model.forward(x_batch, y_batch, train=True, gpu=self.gpu)
                loss.backward()
                self.optimizer.update()

                sum_train_loss += float(loss.data) * real_batchsize
                sum_train_accuracy += float(acc.data) * real_batchsize

            logging.info(
                'train mean loss={}, accuracy={}'.format(
                    sum_train_loss / self.n_train,
                    sum_train_accuracy / self.n_train
                )
            )
            self.train_accuracies.append(sum_train_accuracy / self.n_train)
            self.train_losses.append(sum_train_loss / self.n_train)

            # evalation
            sum_test_accuracy = 0
            sum_test_loss = 0
            for i in xrange(0, self.n_test, batchsize):
                x_batch = self.xp.asarray(self.x_test[i:i+batchsize])
                y_batch = self.xp.asarray(self.y_test[i:i+batchsize])

                real_batchsize = len(x_batch)

                loss, acc = self.model.forward(x_batch, y_batch, train=False, gpu=self.gpu)

                sum_test_loss += float(loss.data) * real_batchsize
                sum_test_accuracy += float(acc.data) * real_batchsize

            logging.info(
                'test mean loss={}, accuracy={}'.format(
                    sum_test_loss / self.n_test,
                    sum_test_accuracy / self.n_test
                )
            )
            self.test_accuracies.append(sum_test_accuracy / self.n_test)
            self.test_losses.append(sum_test_loss / self.n_test)

            epoch += 1

In [18]:
n_outputs = 10
in_channels = 3

data[0].shape = data[0].shape[0], 3, 32, 32
data[1].shape = data[1].shape[0], 3, 32, 32

start_time = time.time()

cnn = CNN(
    data=data,
    target=target,
    gpu=-1,
    in_channels=in_channels,
    n_outputs=n_outputs,
    n_hidden=100
)

cnn.train_and_test(n_epoch=20)

end_time = time.time()

logging.info("time = {} min".format((end_time - start_time) / 60.0))

INFO:root:epoch 1
INFO:root:train mean loss=2.1686071825, accuracy=0.184299999401
INFO:root:test mean loss=1.94828202605, accuracy=0.280000001043
INFO:root:epoch 2
INFO:root:train mean loss=1.88626817703, accuracy=0.299700001031
INFO:root:test mean loss=1.74593402028, accuracy=0.361100000143
INFO:root:epoch 3
INFO:root:train mean loss=1.73112275362, accuracy=0.359900000989
INFO:root:test mean loss=1.6026442039, accuracy=0.415200000107
INFO:root:epoch 4
INFO:root:train mean loss=1.66691746116, accuracy=0.385299998224
INFO:root:test mean loss=1.60664937615, accuracy=0.421899998486
INFO:root:epoch 5
INFO:root:train mean loss=1.58512838244, accuracy=0.417099999189
INFO:root:test mean loss=1.55425390124, accuracy=0.433799996078
INFO:root:epoch 6
INFO:root:train mean loss=1.52956239104, accuracy=0.44069999963
INFO:root:test mean loss=1.50054981828, accuracy=0.445399997532
INFO:root:epoch 7
INFO:root:train mean loss=1.49740957737, accuracy=0.456099997759
INFO:root:test mean loss=1.43055567265